<a href="https://colab.research.google.com/github/isinare/FOSSProjectSem2/blob/main/flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the system tool (required for Linux/Colab)
!apt-get install poppler-utils

# Install the Python library
!pip install pdf2image flask

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
import os
import tempfile
import shutil
import zipfile
from flask import Flask, request, render_template_string, send_file, flash, redirect, after_this_request
from pdf2image import convert_from_path

app = Flask(__name__)
app.secret_key = "super_secret_key"

# --- HTML TEMPLATE (Jinja2) ---
# We store the HTML directly here to make it a single-file app for Colab
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>PDF to JPG Converter</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
    <style>body { font-family: 'Poppins', sans-serif; }</style>
</head>
<body class="bg-gray-900 text-white min-h-screen flex flex-col items-center justify-center p-4">

    <div class="w-full max-w-md bg-gray-800 rounded-xl shadow-2xl border border-gray-700 p-8 text-center">

        <div class="mb-6">
            <div class="w-20 h-20 bg-blue-600 rounded-full flex items-center justify-center mx-auto mb-4 shadow-lg shadow-blue-500/30">
                <i class="fa-solid fa-wand-magic-sparkles text-3xl"></i>
            </div>
            <h1 class="text-3xl font-bold">PDF to JPG</h1>
            <p class="text-gray-400 mt-2 text-sm">Powered by Python & Poppler</p>
        </div>

        <!-- Flash Messages (Jinja2 Logic) -->
        {% with messages = get_flashed_messages() %}
          {% if messages %}
            <div class="mb-4 p-3 bg-red-500/20 border border-red-500 rounded text-red-200 text-sm">
              {{ messages[0] }}
            </div>
          {% endif %}
        {% endwith %}

        <!-- Upload Form -->
        <form action="/convert" method="POST" enctype="multipart/form-data" class="space-y-6" id="uploadForm">

            <label class="block group cursor-pointer">
                <span class="sr-only">Choose PDF</span>
                <div class="w-full h-32 border-2 border-dashed border-gray-600 rounded-lg flex flex-col items-center justify-center group-hover:border-blue-500 group-hover:bg-gray-700/50 transition-all">
                    <i class="fa-solid fa-cloud-arrow-up text-3xl text-gray-500 group-hover:text-blue-400 mb-2"></i>
                    <p class="text-sm text-gray-400 group-hover:text-white">Click to upload PDF</p>
                    <p id="filename" class="text-xs text-blue-400 mt-2 font-mono"></p>
                </div>
                <input type="file" name="pdf_file" accept=".pdf" class="hidden" onchange="updateFilename(this)">
            </label>

            <button type="submit" id="submitBtn" class="w-full bg-gradient-to-r from-blue-600 to-purple-600 hover:from-blue-500 hover:to-purple-500 text-white font-bold py-3 px-8 rounded-lg shadow-lg transform transition active:scale-95 disabled:opacity-50 disabled:cursor-not-allowed">
                Convert Now
            </button>
        </form>

        <!-- Loading State (Hidden by default) -->
        <div id="loading" class="hidden mt-6">
            <i class="fa-solid fa-circle-notch fa-spin text-2xl text-blue-500 mb-2"></i>
            <p class="text-sm text-gray-400">Processing... this might take a moment.</p>
        </div>

    </div>

    <script>
        function updateFilename(input) {
            const display = document.getElementById('filename');
            if (input.files.length > 0) {
                display.textContent = input.files[0].name;
            } else {
                display.textContent = "";
            }
        }

        document.getElementById('uploadForm').onsubmit = function() {
            document.getElementById('uploadForm').classList.add('opacity-50', 'pointer-events-none');
            document.getElementById('loading').classList.remove('hidden');
        };
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    # Use render_template_string to render the HTML string defined above
    return render_template_string(HTML_TEMPLATE)

@app.route('/convert', methods=['POST'])
def convert_pdf():
    if 'pdf_file' not in request.files:
        flash('No file uploaded')
        return redirect('/')

    file = request.files['pdf_file']
    if file.filename == '' or not file.filename.lower().endswith('.pdf'):
        flash('Invalid PDF file')
        return redirect('/')

    # 1. Create Temp Job Directory
    job_dir = tempfile.mkdtemp()

    # 2. Schedule Cleanup (Runs after data is sent to user)
    @after_this_request
    def cleanup(response):
        try:
            shutil.rmtree(job_dir)
            print(f"Deleted temp job: {job_dir}")
        except Exception as e:
            print(f"Error cleaning up: {e}")
        return response

    try:
        # Paths
        input_pdf = os.path.join(job_dir, 'input.pdf')
        output_dir = os.path.join(job_dir, 'images')
        output_zip = os.path.join(job_dir, 'images.zip')

        os.makedirs(output_dir)
        file.save(input_pdf)

        # 3. Perform Conversion (Pure Python/Poppler Wrapper)
        print("Starting PDF conversion...")

        # convert_from_path returns a list of PIL Image objects
        images = convert_from_path(input_pdf)

        if not images:
            flash('Failed to read PDF or PDF is empty.')
            return redirect('/')

        # Save each page as a JPEG
        for i, image in enumerate(images):
            # Create a filename like page_001.jpg, page_002.jpg
            image_filename = f"page_{i + 1:03d}.jpg"
            image_path = os.path.join(output_dir, image_filename)
            image.save(image_path, 'JPEG')

        # 4. Zip Results
        with zipfile.ZipFile(output_zip, 'w') as z:
            for filename in os.listdir(output_dir):
                if filename.endswith('.jpg'):
                    z.write(os.path.join(output_dir, filename), arcname=filename)

        # 5. Send File
        return send_file(
            output_zip,
            mimetype='application/zip',
            as_attachment=True,
            download_name='converted_images.zip'
        )

    except Exception as e:
        print(f"Error: {e}")
        flash('Server error occurred during conversion.')
        return redirect('/')

if __name__ == "__main__":
    # Import pyngrok here to avoid issues if running locally without it installed
    from pyngrok import ngrok

    # --- INSERT AUTH TOKEN HERE ---
    ngrok.set_auth_token("37T07ClkM1o41FzArQeCqDIFYwx_2GtibAaji5w9Yvw6ct6EK")

    # Terminate open tunnels if any exist to prevent errors
    ngrok.kill()

    # Open an HTTP tunnel on port 5000
    public_url = ngrok.connect(5000).public_url
    print(f" * Public URL: {public_url}")

    # Update any base URLs to use the public one
    app.config["BASE_URL"] = public_url

    # Run the app
    app.run(port=5000)

 * Public URL: https://moderately-unmagisterial-janel.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 09:02:39] "GET / HTTP/1.1" 200 -


Starting PDF conversion...


INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 09:06:02] "POST /convert HTTP/1.1" 200 -


Deleted temp job: /tmp/tmpw19ro4jc
